In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
#from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
#from xgboost import XGBRegressor
import pickle
import joblib
#from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor
#import lightgbm as lgb
import os

In [3]:
base_path = r"C:\Users\imate\Documents\24.9.Notebooks_training\Energy-predictor\data\2_processed"
train_file = os.path.join(base_path, "2_fe_train.csv")
test_file = os.path.join(base_path, "2_fe_test.csv")
train_df=pd.read_csv(train_file)
test_df=pd.read_csv(test_file)


In [4]:
base_path = r"C:\Users\imate\Documents\24.9.Notebooks_training\Energy-predictor\data\0_raw"
mtrain_file = os.path.join(base_path, "0_merged_train.csv")
mtest_file = os.path.join(base_path, "0_merged_test.csv")
mtr=pd.read_csv(mtrain_file)
mte=pd.read_csv(mtest_file)

# Log transformation

- square_feet, meter_readings

In [5]:
#train_df['meter_reading']=np.log1p(train_df['meter_reading'])
#train_df['square_feet']=np.log1p(train_df['square_feet'])
#test_df['square_feet']=np.log1p(test_df['square_feet'])

# Label encoding 

- primary_use, season

0- 'Office', 
1- 'Education', 
2- 'Entertainment/public assembly',       
3- 'Public services', 

4- 'Technology/science', 
5- 'Parking',
6-'Food sales and service'

7- 'Lodging/residential', 
8- 'Other', 
9- 'Services', 
10- 'Healthcare'

In [6]:
train_df.primary_use.unique()

array(['Office', 'Education', 'Entertainment/public assembly',
       'Public services', 'Technology/science', 'Parking',
       'Food sales and service'], dtype=object)

In [7]:
train_df.season.unique()

array(['Spring', 'Autumn', 'Summer', 'Winter'], dtype=object)

In [8]:
test_df.primary_use.unique()

array(['Lodging/residential', 'Education',
       'Entertainment/public assembly', 'Other', 'Public services',
       'Office', 'Services', 'Healthcare'], dtype=object)

In [9]:
test_df.season.unique()

array(['Winter', 'Spring', 'Autumn', 'Summer'], dtype=object)

In [10]:
#baseline model
y_pred_df= train_df.groupby(['site_id', 'primary_use'])['meter_reading'].mean().reset_index()
y_pred_df

,site_id,primary_use,meter_reading
0,1,Office,21.300000
1,2,Education,512.745000
2,2,Office,128.370000
3,3,Entertainment/public assembly,29.180000
4,4,Education,9.250000
5,5,Education,5.000000
6,5,Office,20.200000
7,5,Public services,30.700000
8,6,Entertainment/public assembly,2.291900
9,8,Entertainment/public assembly,4.166700


In [11]:
train_df=train_df.sort_values(by='timestamp')
X_train, X_cv= train_test_split(train_df, test_size=0.20, shuffle=False)

#y_pred for baseline
y_pred_df.rename(columns={"meter_reading": "y_pred_base"}, inplace=True)
X_train= X_train.merge(y_pred_df, on=['site_id', 'primary_use'], how='left')
X_cv= X_cv.merge(y_pred_df,on=['site_id', 'primary_use'], how='left')

In [12]:
#rsmle score for baseline model
rmse_base_tr=math.sqrt(mean_squared_error(X_train['meter_reading'], X_train['y_pred_base']))
rmse_base_cv=math.sqrt(mean_squared_error(X_cv['meter_reading'], X_cv['y_pred_base']))
print("RMSE for baseline model for train data:", rmse_base_tr)
print("RMSE for baseline model for cv data:", rmse_base_cv)

RMSE for baseline model for train data: 146.93853536413158
RMSE for baseline model for cv data: 90.50837317728028


In [13]:
#Label encoding primary use variables
from sklearn.preprocessing import LabelEncoder

label_enc= LabelEncoder()
label_enc.fit(train_df['primary_use'])
X_train['primary_use']= label_enc.transform(X_train['primary_use'])
X_cv['primary_use']= label_enc.transform(X_cv['primary_use'])


In [14]:
label_enc.fit(test_df['primary_use'])
test_df['primary_use']= label_enc.transform(test_df['primary_use'])

In [15]:
label_enc.fit(train_df['season'])

X_train['season']= label_enc.transform(X_train['season'])
X_cv['season']= label_enc.transform(X_cv['season'])
test_df['season']= label_enc.transform(test_df['season'])


In [16]:

y_readings_tr=X_train['meter_reading']
y_readings_cv=X_cv['meter_reading']

X_train.drop(['meter_reading', 'y_pred_base'], axis=1, inplace=True)
X_cv.drop(['meter_reading', 'y_pred_base'], axis=1, inplace=True)

# Multicollinearity

- variance inflation factor for all columns

In [17]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
s=X_train._get_numeric_data()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(s.values, i) for i in range(s.shape[1])]
vif["features"] = s.columns

In [18]:

import pyarrow 
X_train.drop(['site_id', 'dew_temperature','timestamp','year','dayofyear','city','country'],axis=1,inplace=True)
X_cv.drop(['site_id', 'dew_temperature','timestamp','year','dayofyear','city','country'],axis=1,inplace=True)
test_df.drop(['site_id', 'dew_temperature','timestamp','year','dayofyear'],axis=1,inplace=True)


#Saving final data in files
test_df.to_feather(r'C:\Users\imate\Documents\24.9.Notebooks_training\Energy-predictor\data\2_processed\2_test.ftr')
#Converting pandas series to numpy
y_readings_tr=y_readings_tr.to_numpy()
y_readings_cv=y_readings_cv.to_numpy()
X_train.to_feather(r'C:\Users\imate\Documents\24.9.Notebooks_training\Energy-predictor\data\2_processed\2_X_train_new.ftr')
X_cv.to_feather(r'C:\Users\imate\Documents\24.9.Notebooks_training\Energy-predictor\data\2_processed\2_X_cv_new.ftr')
np.save(r'C:\Users\imate\Documents\24.9.Notebooks_training\Energy-predictor\reports\y_readings_tr_new.npy', y_readings_tr)
np.save(r'C:\Users\imate\Documents\24.9.Notebooks_training\Energy-predictor\reports\y_readings_cv_new.npy',y_readings_cv)